In [74]:
# Define paths
import os


data_path = "../input"
test_dir = os.path.join(data_path, "test")
submission_path = "../working/submission.csv"

# Model path - adjust if your best model is saved in a different location
model_path = "/kaggle/input/train-yolo/yolo_weights/motor_detector/weights/best.pt"

model_path = "../working/yolo_model/checkpoint_best_ema.pth"

CONCENTRATION = 0.5
TRESHOLD = 0.10
NMS_IOU_THRESHOLD = 0.2

In [75]:
import numpy as np
import pandas as pd
from rfdetr import RFDETRBase
from PIL import Image
import torch
import torchvision.transforms.functional as F
import tqdm
from itertools import chain


def get_tomos(test_dir):
    test_tomos = sorted(
        [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]
    )

    total_tomos = len(test_tomos)
    print(f"Total tomograms: {total_tomos}")
    return test_tomos


def get_slice_files_from_tomo_id(tomo_id, test_dir, concentration):
    tomo_dir = os.path.join(test_dir, tomo_id)
    slice_files = sorted([f for f in os.listdir(tomo_dir) if f.endswith(".jpg")])

    # Apply CONCENTRATION to reduce the number of slices processed
    # This will process approximately CONCENTRATION fraction of all slices
    selected_indices = np.linspace(
        0, len(slice_files) - 1, int(len(slice_files) * concentration)
    )
    selected_indices = np.round(selected_indices).astype(int)
    slice_files = [slice_files[i] for i in selected_indices]
    print(f"Total slices in {tomo_id}: {len(slice_files)}")

    return slice_files


def load_images(slice_files, tomo_dir):
    images = {
        os.path.join(tomo_dir, f): Image.open(os.path.join(tomo_dir, f)).convert("RGB")
        for f in slice_files
    }
    return images


import supervision as sv


def get_center_from_2dbox(box: np.ndarray):
    x1, y1, x2, y2 = box

    # Calculate center coordinates
    x_center = (x1 + x2) / 2
    y_center = (y1 + y2) / 2

    return x_center, y_center


def get_slice_number_from_filename(filename: str):
    # Extract the slice number from the filename
    # Assuming the filename format is "slice_1.jpg", "slice_2.jpg", etc.
    # Adjust the parsing logic based on your actual filename format
    slice_number = int(filename.split("/")[-1].split(".")[0].split("_")[-1])
    return slice_number


def make_center_predictions(detection: sv.Detections, slice_file):
    all_detections = []
    for idx, confidence in enumerate(detection.confidence):
        slice_num = get_slice_number_from_filename(slice_file)
        x_center, y_center = get_center_from_2dbox(detection.xyxy[idx])
        all_detections.append(
            {
                "z": round(slice_num),
                "y": round(y_center),
                "x": round(x_center),
                "confidence": float(confidence),
            }
        )
    return all_detections


def perform_3d_nms(detections, iou_threshold):
    """
    Perform 3D Non-Maximum Suppression on detections to merge nearby motors
    """
    if not detections:
        return []

    # Sort by confidence (highest first)
    detections = sorted(detections, key=lambda x: x["confidence"], reverse=True)

    # List to store final detections after NMS
    final_detections = []

    # Define 3D distance function
    def distance_3d(d1, d2):
        return np.sqrt(
            (d1["z"] - d2["z"]) ** 2
            + (d1["y"] - d2["y"]) ** 2
            + (d1["x"] - d2["x"]) ** 2
        )

    # Maximum distance threshold (based on box size and slice gap)
    box_size = 24  # Same as annotation box size
    distance_threshold = box_size * iou_threshold

    # Process each detection
    while detections:
        # Take the detection with highest confidence
        best_detection = detections.pop(0)
        final_detections.append(best_detection)

        # Filter out detections that are too close to the best detection
        detections = [
            d for d in detections if distance_3d(d, best_detection) > distance_threshold
        ]

    return final_detections


def format_for_submission(tomo_id, final_detections):
    # Sort detections by confidence (highest first)
    final_detections.sort(key=lambda x: x["confidence"], reverse=True)

    # If there are no detections, return NA values
    if not final_detections:
        return {
            "tomo_id": tomo_id,
            "Motor axis 0": -1,
            "Motor axis 1": -1,
            "Motor axis 2": -1,
        }

    # Take the detection with highest confidence
    best_detection = final_detections[0]

    # Return result with integer coordinates
    return {
        "tomo_id": tomo_id,
        "Motor axis 0": round(best_detection["z"]),
        "Motor axis 1": round(best_detection["y"]),
        "Motor axis 2": round(best_detection["x"]),
    }


def save_submission(submissions, submission_path):
    submission_df = pd.DataFrame(submissions)

    # Ensure proper column order
    submission_df = submission_df[
        ["tomo_id", "Motor axis 0", "Motor axis 1", "Motor axis 2"]
    ]

    # Save the submission file
    submission_df.to_csv(submission_path, index=False)

    print(f"\nSubmission complete!")
    print(f"Submission saved to: {submission_path}")

    # Display first few rows of submission
    print("\nSubmission preview:")
    print(submission_df.head())


def run_prediction_on_dir(
    dir,
    model,
    threshold=TRESHOLD,
    concentration=CONCENTRATION,
    submission_path=submission_path,
):
    tomos = get_tomos(dir)

    submissions = []
    for tomo_id in tomos:
        slice_files = get_slice_files_from_tomo_id(tomo_id, test_dir, concentration)
        images = load_images(slice_files, os.path.join(test_dir, tomo_id))

        detections_list = []
        for image in tqdm.tqdm(images.values()):
            detection = model.predict(image, threshold=threshold)
            detections_list.append(detection)

        all_detections = chain.from_iterable(
            make_center_predictions(detection, filename)
            for detection, filename in zip(detections_list, images.keys())
        )
        final_detections = perform_3d_nms(all_detections, NMS_IOU_THRESHOLD)

        submissions.append(format_for_submission(tomo_id, final_detections))

    save_submission(submissions, submission_path)


model = RFDETRBase(
    pretrain_weights=model_path,
)

run_prediction_on_dir(test_dir, model)

num_classes mismatch: pretrain weights has 0 classes, but your model has 90 classes
reinitializing detection head with 0 classes


Loading pretrain weights
Total tomograms: 3
Total slices in tomo_003acc: 250


100%|██████████| 250/250 [00:19<00:00, 12.72it/s]


Total slices in tomo_00e047: 150


100%|██████████| 150/150 [00:04<00:00, 32.92it/s]


Total slices in tomo_01a877: 150


100%|██████████| 150/150 [00:04<00:00, 33.02it/s]


Submission complete!
Submission saved to: ../working/submission.csv

Submission preview:
       tomo_id  Motor axis 0  Motor axis 1  Motor axis 2
0  tomo_003acc           118          1161           881
1  tomo_00e047           175           544           603
2  tomo_01a877           142           638           286


In [ ]:
valid_dir = os.path.join("../working/yolo_dataset/images", "valid/*.jpg")

import glob

glob.glob(valid_dir)

['../working/yolo_dataset/images/valid/tomo_e34af8_z0253_y0359_x0500.jpg', '../working/yolo_dataset/images/valid/tomo_46250a_z0173_y0168_x0336.jpg', '../working/yolo_dataset/images/valid/tomo_b50c0f_z0140_y0710_x0779.jpg', '../working/yolo_dataset/images/valid/tomo_823bc7_z0163_y0356_x0805.jpg', '../working/yolo_dataset/images/valid/tomo_fb08b5_z0102_y0671_x0559.jpg', '../working/yolo_dataset/images/valid/tomo_3e7407_z0261_y0349_x0736.jpg', '../working/yolo_dataset/images/valid/tomo_3e7407_z0255_y0348_x0692.jpg', '../working/yolo_dataset/images/valid/tomo_a75c98_z0110_y0456_x0346.jpg', '../working/yolo_dataset/images/valid/tomo_2b3cdf_z0134_y0173_x0662.jpg', '../working/yolo_dataset/images/valid/tomo_81445c_z0100_y0807_x0233.jpg', '../working/yolo_dataset/images/valid/tomo_a75c98_z0108_y0456_x0346.jpg', '../working/yolo_dataset/images/valid/tomo_ff505c_z0114_y0816_x0678.jpg', '../working/yolo_dataset/images/valid/tomo_f3e449_z0172_y0740_x0715.jpg', '../working/yolo_dataset/images/valid

In [ ]:
valid_dir = os.path.join("", "valid")

run_prediction_on_dir(valid_dir, model)

FileNotFoundError: [Errno 2] No such file or directory: '../input/valid'

In [ ]:
import numpy as np
import pandas as pd
import sklearn.metrics


class ParticipantVisibleError(Exception):
    # If you want an error message to be shown to participants, you must raise the error as a ParticipantVisibleError
    # All other errors will only be shown to the competition host. This helps prevent unintentional leakage of solution data.
    pass


def distance_metric(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    thresh_ratio: float,
    min_radius: float,
):
    coordinate_cols = ["Motor axis 0", "Motor axis 1", "Motor axis 2"]
    label_tensor = solution[coordinate_cols].values.reshape(
        len(solution), -1, len(coordinate_cols)
    )
    predicted_tensor = submission[coordinate_cols].values.reshape(
        len(submission), -1, len(coordinate_cols)
    )
    # Find the minimum euclidean distances between the true and predicted points
    solution["distance"] = np.linalg.norm(label_tensor - predicted_tensor, axis=2).min(
        axis=1
    )
    # Convert thresholds from angstroms to voxels
    solution["thresholds"] = solution["Voxel spacing"].apply(
        lambda x: (min_radius * thresh_ratio) / x
    )
    solution["predictions"] = submission["Has motor"].values
    solution.loc[
        (solution["distance"] > solution["thresholds"])
        & (solution["Has motor"] == 1)
        & (submission["Has motor"] == 1),
        "predictions",
    ] = 0
    return solution["predictions"].values


def score(
    solution: pd.DataFrame, submission: pd.DataFrame, min_radius: float, beta: float
) -> float:
    """
    Parameters:
    solution (pd.DataFrame): DataFrame containing ground truth motor positions.
    submission (pd.DataFrame): DataFrame containing predicted motor positions.

    Returns:
    float: FBeta score.

    Example
    --------
    >>> solution = pd.DataFrame({
    ...     'tomo_id': [0, 1, 2, 3],
    ...     'Motor axis 0': [-1, 250, 100, 200],
    ...     'Motor axis 1': [-1, 250, 100, 200],
    ...     'Motor axis 2': [-1, 250, 100, 200],
    ...     'Voxel spacing': [10, 10, 10, 10],
    ...     'Has motor': [0, 1, 1, 1]
    ... })
    >>> submission = pd.DataFrame({
    ...     'tomo_id': [0, 1, 2, 3],
    ...     'Motor axis 0': [100, 251, 600, -1],
    ...     'Motor axis 1': [100, 251, 600, -1],
    ...     'Motor axis 2': [100, 251, 600, -1]
    ... })
    >>> score(solution, submission, 1000, 2)
    0.3571428571428571
    """

    solution = solution.sort_values("tomo_id").reset_index(drop=True)
    submission = submission.sort_values("tomo_id").reset_index(drop=True)

    filename_equiv_array = (
        solution["tomo_id"].eq(submission["tomo_id"], fill_value=0).values
    )

    if np.sum(filename_equiv_array) != len(solution["tomo_id"]):
        raise ValueError(
            "Submitted tomo_id values do not match the sample_submission file"
        )

    submission["Has motor"] = 1
    # If any columns are missing an axis, it's marked with no motor
    select = (submission[["Motor axis 0", "Motor axis 1", "Motor axis 2"]] == -1).any(
        axis="columns"
    )
    submission.loc[select, "Has motor"] = 0

    cols = ["Has motor", "Motor axis 0", "Motor axis 1", "Motor axis 2"]
    assert all(col in submission.columns for col in cols)

    # Calculate a label of 0 or 1 using the 'has motor', and 'motor axis' values
    predictions = distance_metric(
        solution,
        submission,
        thresh_ratio=1.0,
        min_radius=min_radius,
    )

    return sklearn.metrics.fbeta_score(
        solution["Has motor"].values, predictions, beta=beta
    )

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import cv2
from tqdm.notebook import tqdm
import threading
import time
from contextlib import nullcontext
from concurrent.futures import ThreadPoolExecutor

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Define paths
data_path = "../input"
test_dir = os.path.join(data_path, "test")
submission_path = "../working/submission.csv"

# Model path - adjust if your best model is saved in a different location
model_path = "/kaggle/input/train-yolo/yolo_weights/motor_detector/weights/best.pt"

# Detection parameters
CONFIDENCE_THRESHOLD = 0.45  # Lower threshold to catch more potential motors
MAX_DETECTIONS_PER_TOMO = 3  # Keep track of top N detections per tomogram
NMS_IOU_THRESHOLD = 0.2  # Non-maximum suppression threshold for 3D clustering
CONCENTRATION = 1  # ONLY PROCESS 1/20 slices for fast submission


# GPU profiling context manager
class GPUProfiler:
    def __init__(self, name):
        self.name = name
        self.start_time = None

    def __enter__(self):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        self.start_time = time.time()
        return self

    def __exit__(self, *args):
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        elapsed = time.time() - self.start_time
        print(f"[PROFILE] {self.name}: {elapsed:.3f}s")


# Check GPU availability and set up optimizations
device = "cuda:0" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8  # Default batch size, will be adjusted dynamically if GPU available

if device.startswith("cuda"):
    # Set CUDA optimization flags
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False
    torch.backends.cuda.matmul.allow_tf32 = True  # Allow TF32 on Ampere GPUs
    torch.backends.cudnn.allow_tf32 = True

    # Print GPU info
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9  # Convert to GB
    print(f"Using GPU: {gpu_name} with {gpu_mem:.2f} GB memory")

    # Get available GPU memory and set batch size accordingly
    free_mem = gpu_mem - torch.cuda.memory_allocated(0) / 1e9
    BATCH_SIZE = max(8, min(32, int(free_mem * 4)))  # 4 images per GB as rough estimate
    print(
        f"Dynamic batch size set to {BATCH_SIZE} based on {free_mem:.2f}GB free memory"
    )
else:
    print("GPU not available, using CPU")
    BATCH_SIZE = 4  # Reduce batch size for CPU


def normalize_slice(slice_data):
    """
    Normalize slice data using 2nd and 98th percentiles for better contrast
    """
    p2 = np.percentile(slice_data, 2)
    p98 = np.percentile(slice_data, 98)
    clipped_data = np.clip(slice_data, p2, p98)
    normalized = 255 * (clipped_data - p2) / (p98 - p2)
    return np.uint8(normalized)


def preload_image_batch(file_paths):
    """Preload a batch of images to CPU memory"""
    images = []
    for path in file_paths:
        img = cv2.imread(path)
        if img is None:
            # Try with PIL as fallback
            img = np.array(Image.open(path))
        images.append(img)
    return images


def process_tomogram(tomo_id, model, index=0, total=1):
    """
    Process a single tomogram and return the most confident motor detection
    """
    print(f"Processing tomogram {tomo_id} ({index}/{total})")

    # Get all slice files for this tomogram
    tomo_dir = os.path.join(test_dir, tomo_id)
    slice_files = sorted([f for f in os.listdir(tomo_dir) if f.endswith(".jpg")])

    # Apply CONCENTRATION to reduce the number of slices processed
    # This will process approximately CONCENTRATION fraction of all slices
    selected_indices = np.linspace(
        0, len(slice_files) - 1, int(len(slice_files) * CONCENTRATION)
    )
    selected_indices = np.round(selected_indices).astype(int)
    slice_files = [slice_files[i] for i in selected_indices]

    print(
        f"Processing {len(slice_files)} out of {len(os.listdir(tomo_dir))} slices based on CONCENTRATION={CONCENTRATION}"
    )

    # Create a list to store all detections
    all_detections = []

    # Create CUDA streams for parallel processing if using GPU
    if device.startswith("cuda"):
        streams = [torch.cuda.Stream() for _ in range(min(4, BATCH_SIZE))]
    else:
        streams = [None]

    # Variables for preloading
    next_batch_thread = None
    next_batch_images = None

    # Process slices in batches
    for batch_start in range(0, len(slice_files), BATCH_SIZE):
        # Wait for previous preload thread if it exists
        if next_batch_thread is not None:
            next_batch_thread.join()
            next_batch_images = None

        batch_end = min(batch_start + BATCH_SIZE, len(slice_files))
        batch_files = slice_files[batch_start:batch_end]

        # Start preloading next batch
        next_batch_start = batch_end
        next_batch_end = min(next_batch_start + BATCH_SIZE, len(slice_files))
        next_batch_files = (
            slice_files[next_batch_start:next_batch_end]
            if next_batch_start < len(slice_files)
            else []
        )

        if next_batch_files:
            next_batch_paths = [os.path.join(tomo_dir, f) for f in next_batch_files]
            next_batch_thread = threading.Thread(
                target=preload_image_batch, args=(next_batch_paths,)
            )
            next_batch_thread.start()
        else:
            next_batch_thread = None

        # Split batch across streams for parallel processing
        sub_batches = np.array_split(batch_files, len(streams))
        sub_batch_results = []

        for i, sub_batch in enumerate(sub_batches):
            if len(sub_batch) == 0:
                continue

            stream = streams[i % len(streams)]
            with (
                torch.cuda.stream(stream)
                if stream and device.startswith("cuda")
                else nullcontext()
            ):
                # Process sub-batch
                sub_batch_paths = [
                    os.path.join(tomo_dir, slice_file) for slice_file in sub_batch
                ]
                sub_batch_slice_nums = [
                    int(slice_file.split("_")[1].split(".")[0])
                    for slice_file in sub_batch
                ]

                # Run inference with profiling
                with GPUProfiler(f"Inference batch {i+1}/{len(sub_batches)}"):
                    sub_results = model(sub_batch_paths, verbose=False)

                # Process each result in this sub-batch
                for j, result in enumerate(sub_results):
                    if len(result.boxes) > 0:
                        boxes = result.boxes
                        for box_idx, confidence in enumerate(boxes.conf):
                            if confidence >= CONFIDENCE_THRESHOLD:
                                # Get bounding box coordinates
                                x1, y1, x2, y2 = boxes.xyxy[box_idx].cpu().numpy()

                                # Calculate center coordinates
                                x_center = (x1 + x2) / 2
                                y_center = (y1 + y2) / 2

                                # Store detection with 3D coordinates
                                all_detections.append(
                                    {
                                        "z": round(sub_batch_slice_nums[j]),
                                        "y": round(y_center),
                                        "x": round(x_center),
                                        "confidence": float(confidence),
                                    }
                                )

        # Synchronize streams
        if device.startswith("cuda"):
            torch.cuda.synchronize()

    # Clean up thread if still running
    if next_batch_thread is not None:
        next_batch_thread.join()

    # 3D Non-Maximum Suppression to merge nearby detections across slices
    final_detections = perform_3d_nms(all_detections, NMS_IOU_THRESHOLD)

    # Sort detections by confidence (highest first)
    final_detections.sort(key=lambda x: x["confidence"], reverse=True)

    # If there are no detections, return NA values
    if not final_detections:
        return {
            "tomo_id": tomo_id,
            "Motor axis 0": -1,
            "Motor axis 1": -1,
            "Motor axis 2": -1,
        }

    # Take the detection with highest confidence
    best_detection = final_detections[0]

    # Return result with integer coordinates
    return {
        "tomo_id": tomo_id,
        "Motor axis 0": round(best_detection["z"]),
        "Motor axis 1": round(best_detection["y"]),
        "Motor axis 2": round(best_detection["x"]),
    }


def perform_3d_nms(detections, iou_threshold):
    """
    Perform 3D Non-Maximum Suppression on detections to merge nearby motors
    """
    if not detections:
        return []

    # Sort by confidence (highest first)
    detections = sorted(detections, key=lambda x: x["confidence"], reverse=True)

    # List to store final detections after NMS
    final_detections = []

    # Define 3D distance function
    def distance_3d(d1, d2):
        return np.sqrt(
            (d1["z"] - d2["z"]) ** 2
            + (d1["y"] - d2["y"]) ** 2
            + (d1["x"] - d2["x"]) ** 2
        )

    # Maximum distance threshold (based on box size and slice gap)
    box_size = 24  # Same as annotation box size
    distance_threshold = box_size * iou_threshold

    # Process each detection
    while detections:
        # Take the detection with highest confidence
        best_detection = detections.pop(0)
        final_detections.append(best_detection)

        # Filter out detections that are too close to the best detection
        detections = [
            d for d in detections if distance_3d(d, best_detection) > distance_threshold
        ]

    return final_detections


def generate_submission():
    """
    Main function to generate the submission file
    """
    # Get list of test tomograms
    test_tomos = sorted(
        [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]
    )
    total_tomos = len(test_tomos)

    print(f"Found {total_tomos} tomograms in test directory")

    # Debug image loading for the first tomogram
    if test_tomos:
        debug_image_loading(test_tomos[0])

    # Clear GPU cache before starting
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Initialize model once outside the processing loop
    print(f"Loading YOLO model from {model_path}")
    model = YOLO(model_path)
    model.to(device)

    # Additional optimizations for inference
    if device.startswith("cuda"):
        # Fuse conv and bn layers for faster inference
        model.fuse()

        # Enable model half precision (FP16) if on compatible GPU
        if torch.cuda.get_device_capability(0)[0] >= 7:  # Volta or newer
            model.model.half()
            print("Using half precision (FP16) for inference")

    # Process tomograms with parallelization
    results = []
    motors_found = 0

    # Using ThreadPoolExecutor with max_workers=1 since each worker uses the GPU already
    # and we're parallelizing within each tomogram processing
    with ThreadPoolExecutor(max_workers=1) as executor:
        future_to_tomo = {}

        # Submit all tomograms for processing
        for i, tomo_id in enumerate(test_tomos, 1):
            future = executor.submit(process_tomogram, tomo_id, model, i, total_tomos)
            future_to_tomo[future] = tomo_id

        # Process completed futures as they complete
        for future in future_to_tomo:
            tomo_id = future_to_tomo[future]
            try:
                # Clear CUDA cache between tomograms
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

                result = future.result()
                results.append(result)

                # Update motors found count
                has_motor = not pd.isna(result["Motor axis 0"])
                if has_motor:
                    motors_found += 1
                    print(
                        f"Motor found in {tomo_id} at position: "
                        f"z={result['Motor axis 0']}, y={result['Motor axis 1']}, x={result['Motor axis 2']}"
                    )
                else:
                    print(f"No motor detected in {tomo_id}")

                print(
                    f"Current detection rate: {motors_found}/{len(results)} ({motors_found/len(results)*100:.1f}%)"
                )

            except Exception as e:
                print(f"Error processing {tomo_id}: {e}")
                # Create a default entry for failed tomograms
                results.append(
                    {
                        "tomo_id": tomo_id,
                        "Motor axis 0": -1,
                        "Motor axis 1": -1,
                        "Motor axis 2": -1,
                    }
                )

    # Create submission dataframe
    submission_df = pd.DataFrame(results)

    # Ensure proper column order
    submission_df = submission_df[
        ["tomo_id", "Motor axis 0", "Motor axis 1", "Motor axis 2"]
    ]

    # Save the submission file
    submission_df.to_csv(submission_path, index=False)

    print(f"\nSubmission complete!")
    print(
        f"Motors detected: {motors_found}/{total_tomos} ({motors_found/total_tomos*100:.1f}%)"
    )
    print(f"Submission saved to: {submission_path}")

    # Display first few rows of submission
    print("\nSubmission preview:")
    print(submission_df.head())

    return submission_df